<a href="https://colab.research.google.com/github/oItsMineZ/RVC-v2-AICover-Colab/blob/main/oItsMineZ-rvc-v2-AICover-Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **oItsMineZ RVC v2 Colab (AI Cover Edition 🎙)**
- [![oItsMineZ's RVC Model](https://img.shields.io/badge/Original_notebook-_CoverGen_No_UI.ipynb-black?style=for-the-badge&logo=github&logoColor=white)](https://github.com/ardha27/AICoverGen-NoUI-Colab/blob/main/CoverGen_No_UI.ipynb)
- [![Tutorial Video](https://img.shields.io/badge/Youtube-_Tutorial-FF0000?style=for-the-badge&logo=youtube&logoColor=white)]()
- [![oItsMineZ's RVC Model](https://img.shields.io/badge/%F0%9F%A4%97_Hugging_Face-_oItsMineZ's%20RVC%20%20Model-yellow?style=for-the-badge&logoColor=yellow)](https://huggingface.co/oItsMineZ/oItsMineZ-RVC-Model)

## **Follow Me 💎**
- 🔴 Youtube

> [![oItsMineZ on Youtube](https://img.shields.io/badge/YouTube-FF0000?style=for-the-badge&logo=youtube&logoColor=white)](bit.ly/oItsMineZ)

- 📱 Tiktok

> [![oItsMineZ on Tiktok](https://img.shields.io/badge/TikTok-000000?style=for-the-badge&logo=tiktok&logoColor=white)](https://www.tiktok.com/@oitsminez)





## **ข้อควรรู้**

- RVC v2 (Retrieval Based Voice Conversion v2) เป็น AI Voice Model ที่ปรับปรุงมาจาก VITS ที่ทำให้เทรนโมเดลได้ง่ายขึ้น และคุณภาพของเสียงดีขึ้น
- Notebook นี้ใช้สำหรับเฉพาะ **AI Cover** เพลงเท่านั้น! ถ้าอยากใช้เฉพาะเสียงพูดให้ใช้ [**ตัวนี้แทน**](https://huggingface.co/spaces/oItsMineZ/RVC-v2-WebUI)
- บางเพลงอาจ**ใช้เวลานานมากๆ** ขึ้นอยู่กับความยาวหรือขนาดไฟล์เพลง (จากที่ผมลองเพลงของ [**OneRepublic - Counting Stars**](https://youtu.be/hT_nvWreIhg) ใช้เวลา 55 นาที 😱)

## **ขั้นตอนเบื้องต้น**

**ใช้ไฟล์เสียงของตัวเอง**
- สร้างโฟเดอร์ใน Google Drive ในบัญชีที่ใช้ Google Colab ชื่อว่า ***oitsminez-rvc-v2***
- ในโฟเดอร์ ***oitsminez-rvc-v2*** ให้สร้างโฟเดอร์ชื่อว่า ***song_input*** แล้วอัปโหลดไฟล์เสียง (ที่มีเฉพาะ Vocal) ***นามสกุล .wma เท่านั้น*** ลงในโฟเดอร์ ***oitsminez-rvc-v2***

**ใช้ไฟล์เสียงจาก YouTube**
- คัดลอกลิงก์วิดีโอที่เราต้องการมาใส่ในช่อง **[Youtube]** ด้านล่างได้เลย

In [ ]:
#@title **Installation (รัน Cell นี้ก่อนใช้)**

#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

#@title Create Directory
import os
os.makedirs('/content/drive/MyDrive/oitsminez-rvc-v2', exist_ok=True)
os.makedirs('/content/drive/MyDrive/oitsminez-rvc-v2/song_input', exist_ok=True)
os.makedirs('/content/drive/MyDrive/oitsminez-rvc-v2/song_output', exist_ok=True)

#@title Clone repository
!git clone https://github.com/oItsMineZ/AICoverGen-Mod.git
%cd AICoverGen-Mod

#@title Install requirements
!pip install -q -r requirements.txt
!sudo apt update
!sudo apt install sox

#@title Download MDXNet Vocal Separation and Hubert Base Models
!python src/download_models.py

## Use Model

In [ ]:
# @title Model Download Function

import os
import zipfile
import shutil
import urllib.request

BASE_DIR = os.getcwd()
rvc_models_dir = os.path.join(BASE_DIR, 'rvc_models')

def extract_zip(extraction_folder, zip_name):
    os.makedirs(extraction_folder)
    with zipfile.ZipFile(zip_name, 'r') as zip_ref:
        zip_ref.extractall(extraction_folder)
    os.remove(zip_name)
    index_filepath, model_filepath = None, None
    for root, dirs, files in os.walk(extraction_folder):
        for name in files:
            if name.endswith('.index') and os.stat(os.path.join(root, name)).st_size > 1024 * 100:
                index_filepath = os.path.join(root, name)
            if name.endswith('.pth') and os.stat(os.path.join(root, name)).st_size > 1024 * 1024 * 40:
                model_filepath = os.path.join(root, name)
    if not model_filepath:
        raise Exception(f'No .pth model file was found in the extracted zip. Please check {extraction_folder}.')
    os.rename(model_filepath, os.path.join(extraction_folder, os.path.basename(model_filepath)))
    if index_filepath:
        os.rename(index_filepath, os.path.join(extraction_folder, os.path.basename(index_filepath)))
    for filepath in os.listdir(extraction_folder):
        if os.path.isdir(os.path.join(extraction_folder, filepath)):
            shutil.rmtree(os.path.join(extraction_folder, filepath))

def download_online_model(url, dir_name):
    try:
        print(f'[~] Downloading voice model with name {dir_name}...')
        zip_name = url.split('/')[-1]
        extraction_folder = os.path.join(rvc_models_dir, dir_name)
        if 'pixeldrain.com' in url:
            url = f'https://pixeldrain.com/api/file/{zip_name}'
        urllib.request.urlretrieve(url, zip_name)
        print('[~] Extracting zip...')
        extract_zip(extraction_folder, zip_name)
        print(f'[+] {dir_name} Model successfully downloaded!')
    except Exception as e:
        if os.path.exists(extraction_folder):
            if Select == "Model List":
              if Model_List in List:
                dir_name = List[Model_List]["dir_name"]
                RVC_DIRNAME = List[Model_List]["dir_name"]
            elif Select == "Download Other Model":
              RVC_DIRNAME = {dir_name}

# @title Choose Model
# @markdown เลือก Mode - (***Model List*** คือเลือก Model ที่มีไว้ให้ ส่วน ***Download Other Model*** คือ Download โมเดลอี่นๆมาใช้แทน)
Select = "Model List"  # @param ["Model List", "Download Other Model"]

# @markdown **[Model List]** เลือก Model ที่ต้องการ
Model_List = "DaengGuitar"  # @param ["DaengGuitar", "TAE", "ท่านศาสดา"]

List = {
    "DaengGuitar": {
        "dir_name": "DaengGuitar",
        "url": "https://huggingface.co/oItsMineZ/oItsMineZ-RVC-Model/resolve/main/DaengGuitar/DaengGuitar.zip?download=true",
    },
    "TAE": {
        "dir_name": "TAEEXZENFIRE",
        "url": "https://huggingface.co/oItsMineZ/oItsMineZ-RVC-Model/resolve/main/TAEEXZENFIRE/TAEEXZENFIRE.zip?download=true",
    },
    "ท่านศาสดา": {
        "dir_name": "Sadsada",
        "url": "https://huggingface.co/oItsMineZ/oItsMineZ-RVC-Model/resolve/main/Sadsada/Sadsada.zip?download=true",
    }
}

if Select == "Model List":
    if Model_List in List:
        dir_name = List[Model_List]["dir_name"]
        RVC_DIRNAME = List[Model_List]["dir_name"]
        url = List[Model_List]["url"]
elif Select == "Download Other Model":
    #@markdown **[Download Other Model]** สำหรับใช้ Model อื่นๆ ที่ไม่มีใน List (เฉพาะไฟล์ .zip เท่านั้น!)
    url = "https://pixeldrain.com/u/3tJmABXA"  # @param {type:"string"}
    dir_name = "Gura"  # @param {type:"string"}

download_online_model(url, dir_name)

In [ ]:
# @title Generate | Output generated inside "drive\MyDrive\oitsminez-rvc-v2\song_output\random_number"

# @markdown เลือก Mode - (***Audio File*** คือใช้ไฟล์เสียงใน Google Drive ของเรา ส่วน ***Youtube*** คือใช้ไฟล์เสียงจาก Youtube โดยตรง)
Select = "Audio File"  # @param ["Audio File", "Youtube"]

if Select == "Audio File":
    #@markdown **[Audio File]** ใส่ชื่อเดี่ยวกับไฟล์เสียงที่อัปโหลดใน Google Drive เช่น Vocal.wav ให้ใส่ "Vocal.wav"
    VOCAL_NAME = "Vocal.wav" #@param {type: "string"}
    VOCAL_PATH = '/content/drive/MyDrive/oitsminez-rvc-v2/song_input/'
    SONG_INPUT = os.path.join(VOCAL_PATH, VOCAL_NAME)
elif Select == "Youtube":
    #@markdown **[Youtube]** ใส่ลิงก์ยูทูป
    SONG_INPUT = "https://www.youtube.com/watch?v=jgCVkQhlScc&pp=ygUFY3VwaWQ%3D" # @param {type:"string"}

# @markdown Main Option
PITCH_CHANGE = 0 # @param {type:"integer"}
PITCH_CHANGE_ALL = 0 # @param {type:"integer"}
# @markdown Voice Conversion Options
INDEX_RATE = 0.5 # @param {type:"number"}
FILTER_RADIUS = 3 # @param {type:"integer"}
PITCH_DETECTION_ALGO = "rmvpe" # @param ["rmvpe", "mangio-crepe"]
CREPE_HOP_LENGTH = 128 # @param {type:"integer"}
PROTECT = 0.33 # @param {type:"number"}
REMIX_MIX_RATE = 0.25  # @param {type:"number"}
# @markdown Audio Mixing Options
MAIN_VOL = 0 # @param {type:"integer"}
BACKUP_VOL = 0 # @param {type:"integer"}
INST_VOL = 0 # @param {type:"integer"}
# @markdown Reverb Control
REVERB_SIZE = 0.15 # @param {type:"number"}
REVERB_WETNESS = 0.2 # @param {type:"number"}
REVERB_DRYNESS = 0.8 # @param {type:"number"}
REVERB_DAMPING = 0.7 # @param {type:"number"}
# @markdown Output Format
OUTPUT_FORMAT = "mp3" # @param ["mp3", "wav"]

import subprocess

command = [
    "python",
    "src/main.py",
    "-i", SONG_INPUT,
    "-dir", RVC_DIRNAME,
    "-p", str(PITCH_CHANGE),
    "-k",
    "-ir", str(INDEX_RATE),
    "-fr", str(FILTER_RADIUS),
    "-rms", str(REMIX_MIX_RATE),
    "-palgo", PITCH_DETECTION_ALGO,
    "-hop", str(CREPE_HOP_LENGTH),
    "-pro", str(PROTECT),
    "-mv", str(MAIN_VOL),
    "-bv", str(BACKUP_VOL),
    "-iv", str(INST_VOL),
    "-pall", str(PITCH_CHANGE_ALL),
    "-rsize", str(REVERB_SIZE),
    "-rwet", str(REVERB_WETNESS),
    "-rdry", str(REVERB_DRYNESS),
    "-rdamp", str(REVERB_DAMPING),
    "-oformat", OUTPUT_FORMAT
]

# Open a subprocess and capture its output
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

# Print the output in real-time
for line in process.stdout:
    print(line, end='')

# Wait for the process to finish
process.wait()